# Progetto Computer Science  
## *Siragusa Elisa* , Matricola 075444



https://github.com/gdv/foundationsCS/blob/master/progetti/students/progetto-2021.ipynb


In [1]:
#Import delle Librerie

import pandas as pd
import gzip
import re



In [2]:
#Import dei dataset

Books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression='gzip',
                   sep=";",encoding = 'latin-1',low_memory=False)
BookRatings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression='gzip',
                   sep=";",encoding = 'latin-1',dtype={"User-ID":str})
GoodBooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression='gzip',
                   sep=",",encoding = 'latin-1',dtype={"original_publication_year":int})
Users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression='gzip',
                   sep=";",encoding = 'latin-1')
GoodBooksRatings= pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression='gzip',
                   sep=",",encoding = 'latin-1')





# 
# Question 1: Normalize the location field of Users dataset, splitting into city, region, country.



### Problemi del Dataset

Nel dataset di sono molte disomogeneità nel campo Location. 



In [3]:
#Esempi di righe con problemi
Users.iloc[[435,12345,134376,241302,278808,278847]]

,User-ID,Location,Age
435,436,"ivanhoe, melbourne, victoria, australia",53.0
12345,12346,"&#305;stanbul, istanbul, turkey",19.0
134376,134377,lawrenceville,30.0
241302,241303,"nmrl, addl-ambernath, thane, mumbai, pin 421 5...",35.0
278808,278809,"norfolk, england, england, united kingdom",49.0
278847,278848,"kÃ¶ln, nordrhein-westfalen, germany",NaN


In [4]:
#Split del campo Location
Users["LocationSplit"]=Users["Location"].str.split(r",\s*")

In [5]:
#Dati in cui ci sono meno di tre informazioni
Users[Users["LocationSplit"].str.len()<=2]

,User-ID,Location,Age,LocationSplit
29419,29420,"st. louis, missouri",68.0,"[st. louis, missouri]"
134376,134377,lawrenceville,30.0,[lawrenceville]


In [6]:
#Dati in cui ci sono più di tre informazioni
Users[Users["LocationSplit"].str.len()>3]

,User-ID,Location,Age,LocationSplit
435,436,"ivanhoe, melbourne, victoria, australia",53.0,"[ivanhoe, melbourne, victoria, australia]"
814,815,"davis bay, sechelt, british columbia, canada",43.0,"[davis bay, sechelt, british columbia, canada]"
840,841,"saint charles, mo, missouri, usa",24.0,"[saint charles, mo, missouri, usa]"
956,957,"santiago, santiago,metropolitana, chile",NaN,"[santiago, santiago, metropolitana, chile]"
1459,1460,"mansoura, dakahlia, mansoura, egypt",34.0,"[mansoura, dakahlia, mansoura, egypt]"
...,...,...,...,...
277717,277718,"colorado springs, co 80901, colorado, usa",62.0,"[colorado springs, co 80901, colorado, usa]"
277753,277754,"north bend,, oregon, usa",60.0,"[north bend, , oregon, usa]"
278028,278029,"mansfield, nottinghamshire, england, united ki...",31.0,"[mansfield, nottinghamshire, england, united k..."
278214,278215,"nelson, motueka, nz, new zealand",28.0,"[nelson, motueka, nz, new zealand]"


### Normalizzazione del campo Location

In [7]:
# Funzione per estrarre Country, Region, City da Location 
def EstraiLoc(r,i):
    '''
    Funzione per estrarre Country, Region, City dal campo Location
    i = 0 restituisce City
    i = 1 restituisce Region
    i = 2 restituisce Country
    '''
    loc = r["LocationSplit"]
    l=len(loc)
    if l==3:
        out = [loc[0],loc[1],loc[2]]                
    elif l>3:
        out = ["","",loc[-1]] 
    else:  
        out =["","",""]
    return out[i]




In [8]:
Users["City"]=Users.apply(EstraiLoc,args=[0],axis=1)
Users["Region"]=Users.apply(EstraiLoc,args=[1],axis=1)
Users["Country"]=Users.apply(EstraiLoc,args=[2],axis=1)

Users

,User-ID,Location,Age,LocationSplit,City,Region,Country
0,1,"nyc, new york, usa",NaN,"[nyc, new york, usa]",nyc,new york,usa
1,2,"stockton, california, usa",18.0,"[stockton, california, usa]",stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,"[moscow, yukon territory, russia]",moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,"[porto, v.n.gaia, portugal]",porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,"[farnborough, hants, united kingdom]",farnborough,hants,united kingdom
...,...,...,...,...,...,...,...
278853,278854,"portland, oregon, usa",NaN,"[portland, oregon, usa]",portland,oregon,usa
278854,278855,"tacoma, washington, united kingdom",50.0,"[tacoma, washington, united kingdom]",tacoma,washington,united kingdom
278855,278856,"brampton, ontario, canada",NaN,"[brampton, ontario, canada]",brampton,ontario,canada
278856,278857,"knoxville, tennessee, usa",NaN,"[knoxville, tennessee, usa]",knoxville,tennessee,usa


# 
# Question 2 : For each book in the Books dataset, compute its average rating.

In [9]:
#Left Join dataframe Books e BookRatings (che contiene i valori Book-Rating)

BooksAndRatings=pd.merge(Books,BookRatings,how="left",left_on="ISBN",right_on="ISBN")
BooksAndRatings[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'User-ID','Book-Rating']].sort_values("Book-Title").head(20)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating
502915,0590567330,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,1999,Hyperion Books for Children,18995,0.0
502918,0590567330,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,1999,Hyperion Books for Children,96448,9.0
502917,0590567330,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,1999,Hyperion Books for Children,55927,0.0
502916,0590567330,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,1999,Hyperion Books for Children,35859,0.0
1016285,0964147726,Always Have Popsicles,Rebecca Harvin,1994,Rebecca L. Harvin,172742,0.0
834493,0942320093,Apple Magic (The Collector's series),Martina Boudreau,1984,Amer Cooking Guild,198711,0.0
1032245,0310232546,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,2001,Zonderkidz,269557,8.0
416940,0962295701,Beyond IBM: Leadership Marketing and Finance ...,Lou Mobley,1989,"Teleonet, Incorporated",11601,0.0
962922,0439188970,Clifford Visita El Hospital (Clifford El Gran...,Norman Bridwell,2000,Scholastic,216683,0.0
817851,0399151788,Dark Justice,Jack Higgins,2004,Putnam Publishing Group,98391,10.0


In [10]:
#Average Rating per book (il libro si intende identificato da "ISBN")
AVGRatingBooks=BooksAndRatings.groupby(["ISBN",'Book-Title'],as_index=False)["Book-Rating"].mean().round(2).sort_values("Book-Title").rename(columns={"Book-Rating":"AVG BookRatings"})
AVGRatingBooks

,ISBN,Book-Title,AVG BookRatings
128083,0590567330,A Light in the Storm: The Civil War Diary of ...,2.25
214738,0964147726,Always Have Popsicles,0.00
212879,0942320093,Apple Magic (The Collector's series),0.00
30279,0310232546,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
214115,0962295701,Beyond IBM: Leadership Marketing and Finance ...,0.00
...,...,...,...
259675,3499232499,Ã?Ã?lpiraten.,0.00
252885,325721538X,Ã?Ã?rger mit Produkt X. Roman.,5.25
257095,3451274973,Ã?Ã?sterlich leben.,7.00
256795,3442725739,Ã?Ã?stlich der Berge.,2.67


# 
# Question 3 : For each book in the GoodBooks dataset, compute its average rating

In [11]:
#AverageRating nel dataframe GoodBooks
GoodBooks[["book_id","isbn",'authors',"original_title","average_rating"]]

,book_id,isbn,authors,original_title,average_rating
0,1,439023483,Suzanne Collins,The Hunger Games,4.34
1,2,439554934,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Philosopher's Stone,4.44
2,3,316015849,Stephenie Meyer,Twilight,3.57
3,4,61120081,Harper Lee,To Kill a Mockingbird,4.25
4,5,743273567,F. Scott Fitzgerald,The Great Gatsby,3.89
...,...,...,...,...,...
94,95,375751513,"Oscar Wilde, Jeffrey Eugenides",The Picture of Dorian Gray,4.06
95,96,345803507,E.L. James,Fifty Shades Freed,3.88
96,97,393970124,"Bram Stoker, Nina Auerbach, David J. Skal",Dracula,3.98
97,98,307269981,"Stieg Larsson, Reg Keeland",Flickan som lekte med elden,4.22


In [12]:
#AverageRating calcolato dal dataframe GoodBooksRatings
GoodBooksAndRatings=pd.merge(GoodBooks,GoodBooksRatings,how="left",left_on="book_id",right_on="book_id")
AVGGoodBooksRatings=GoodBooksAndRatings.groupby(["book_id","isbn","authors","original_title","average_rating"],as_index=False)[["rating"]].mean()
AVGGoodBooksRatings.rename(columns={"rating":"AVG GoodBooksRatings"},inplace=True)
AVGGoodBooksRatings

,book_id,isbn,authors,original_title,average_rating,AVG GoodBooksRatings
0,1,439023483,Suzanne Collins,The Hunger Games,4.34,NaN
1,2,439554934,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Philosopher's Stone,4.44,5.0
2,3,316015849,Stephenie Meyer,Twilight,3.57,NaN
3,4,61120081,Harper Lee,To Kill a Mockingbird,4.25,NaN
4,5,743273567,F. Scott Fitzgerald,The Great Gatsby,3.89,4.0
...,...,...,...,...,...,...
93,95,375751513,"Oscar Wilde, Jeffrey Eugenides",The Picture of Dorian Gray,4.06,NaN
94,96,345803507,E.L. James,Fifty Shades Freed,3.88,NaN
95,97,393970124,"Bram Stoker, Nina Auerbach, David J. Skal",Dracula,3.98,NaN
96,98,307269981,"Stieg Larsson, Reg Keeland",Flickan som lekte med elden,4.22,NaN


# 
# Question 4: Merge together all rows sharing the same book title, author and publisher. We will call the resulting dataset merged books. The books that have not been merged together will not appear in merged books.

In [13]:
#DuplicatedBooks contiene tutti i libri che condividono la stessa tupla ['Book-Title', 'Book-Author','Publisher']
DuplicatedBooks=Books[Books.duplicated(['Book-Title', 'Book-Author','Publisher'],keep=False)].sort_values('Book-Title')



#MergedBooks: dal df DuplicatedBooks vengono eliminati i duplicati
MergedBooks=DuplicatedBooks.drop_duplicates(['Book-Title', 'Book-Author','Publisher'])[['Book-Title', 'Book-Author',  'Publisher']]
MergedBooks

,Book-Title,Book-Author,Publisher
156341,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly
140618,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic
179167,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books
213443,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books
94118,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group
...,...,...,...
144921,Zia,Scott O'Dell,Yearling Books
70890,Zia,Scott O'Dell,Laurel-Leaf Books
118930,Zimmermann Telegram,Barbara Tuchman,Ballantine Books
117469,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics


# 
# Question 5: For each book in merged books compute its average rating.

In [14]:
DuplicatedAndRatings = pd.merge(DuplicatedBooks,BookRatings,how="left",left_on="ISBN",right_on="ISBN")

In [15]:
DuplicatedAndRatings.groupby(['Book-Title', 'Book-Author','Publisher']).mean().round(2)

Book-Rating
Book-Title                                         Book-Author             Publisher                                  
!%@ (A Nutshell handbook)                          Donnalyn Frey           O'Reilly                               3.00
'A Hell of a Place to Lose a Cow': An American ... Tim Brookes             National Geographic                    1.89
10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller Barnes &amp; Nobles Books              7.00
101 Famous Poems                                   Roy J. Cook             McGraw-Hill/Contemporary Books         3.83
15 Houseplants Even You Can't Kill                 Joe Elder               Berkley Pub Group                      0.00
...                                                                                                                ...
Zia                                                Scott O'Dell            Laurel-Leaf Books                      2.62
                                                                           Yearling Books                         1.64
Zimmermann Telegram                                Barbara Tuchman         Ballantine Books                       2.00
Zoids Chaotic Century (Zoids: Chaotic Century (... Michiro Ueyama          Viz Comics                            10.00
Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.         RÃ?ÃÂ¼diger Nehberg   Piper                                  3.25

[4725 rows x 1 columns]

# 
# Quesito 6: For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.

In [16]:
#Average ratings per book (ISBN)
AVGBookRatingMerged= DuplicatedAndRatings.groupby(['ISBN','Book-Title', 'Book-Author','Publisher'],as_index=False).mean().sort_values(['Book-Title', 'Book-Author','Publisher']).rename(columns={"Book-Rating":"AVG-Rating"})
AVGBookRatingMerged

,ISBN,Book-Title,Book-Author,Publisher,AVG-Rating
8748,1565920317,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.000000
8749,1565920465,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,0.000000
7481,0792276833,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,0.000000
7486,0792277295,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,3.400000
8755,1566196256,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.250000
...,...,...,...,...,...
8774,1569317666,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000
8775,1569317674,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000
8776,1569317682,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000
9700,3492218091,Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.,RÃ?ÃÂ¼diger Nehberg,Piper,5.000000


In [17]:
AVGBookRatingMerged.groupby(['Book-Title', 'Book-Author','Publisher']).agg({"AVG-Rating":[min,max]})

AVG-Rating  \
                                                                                                                 min   
Book-Title                                         Book-Author             Publisher                                   
!%@ (A Nutshell handbook)                          Donnalyn Frey           O'Reilly                         0.000000   
'A Hell of a Place to Lose a Cow': An American ... Tim Brookes             National Geographic              0.000000   
10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller Barnes &amp; Nobles Books        6.666667   
101 Famous Poems                                   Roy J. Cook             McGraw-Hill/Contemporary Books   0.000000   
15 Houseplants Even You Can't Kill                 Joe Elder               Berkley Pub Group                0.000000   
...                                                                                                              ...   
Zia                                                Scott O'Dell            Laurel-Leaf Books                1.666667   
                                                                           Yearling Books                   0.000000   
Zimmermann Telegram                                Barbara Tuchman         Ballantine Books                 0.000000   
Zoids Chaotic Century (Zoids: Chaotic Century (... Michiro Ueyama          Viz Comics                      10.000000   
Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.         RÃ?ÃÂ¼diger Nehberg   Piper                            2.666667   

                                                                                                                      
                                                                                                                 max  
Book-Title                                         Book-Author             Publisher                                  
!%@ (A Nutshell handbook)                          Donnalyn Frey           O'Reilly                         6.000000  
'A Hell of a Place to Lose a Cow': An American ... Tim Brookes             National Geographic              3.400000  
10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller Barnes &amp; Nobles Books        7.250000  
101 Famous Poems                                   Roy J. Cook             McGraw-Hill/Contemporary Books   5.000000  
15 Houseplants Even You Can't Kill                 Joe Elder               Berkley Pub Group                0.000000  
...                                                                                                              ...  
Zia                                                Scott O'Dell            Laurel-Leaf Books                3.428571  
                                                                           Yearling Books                   2.000000  
Zimmermann Telegram                                Barbara Tuchman         Ballantine Books                 4.000000  
Zoids Chaotic Century (Zoids: Chaotic Century (... Michiro Ueyama          Viz Comics                      10.000000  
Ã?Ã?berleben in der WÃ?ÃÂ¼ste Danakil.         RÃ?ÃÂ¼diger Nehberg   Piper                            5.000000  

[4725 rows x 2 columns]

# 
# Question 7: For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text.

Nel Dataframe *GoodBooks* vengono create due nuove colonne: 

**AuthorsSplit:** contiene la lista dei diversi autori del libro.

**TextReviewsSplit** *(work_text_reviews_count/numero autori)*:contiene il numero di reviews testuali (reviews with a text) per ogni autore del libro assumendo di suddividere equamente il valore della colonna "work_text_reviews_count" per il numero degli autori 

In [18]:
GoodBooks["AuthorsSplit"]=GoodBooks["authors"].str.split(r",\s*")
GoodBooks["TextReviewsSplit"]=round(GoodBooks["work_text_reviews_count"]/GoodBooks["AuthorsSplit"].apply(len))
GoodBooks[["authors","AuthorsSplit","work_text_reviews_count","TextReviewsSplit",'original_publication_year']]

,authors,AuthorsSplit,work_text_reviews_count,TextReviewsSplit,original_publication_year
0,Suzanne Collins,[Suzanne Collins],155254,155254.0,2008
1,"J.K. Rowling, Mary GrandPrÃ©","[J.K. Rowling, Mary GrandPrÃ©]",75867,37934.0,1997
2,Stephenie Meyer,[Stephenie Meyer],95009,95009.0,2005
3,Harper Lee,[Harper Lee],72586,72586.0,1960
4,F. Scott Fitzgerald,[F. Scott Fitzgerald],51992,51992.0,1925
...,...,...,...,...,...
94,"Oscar Wilde, Jeffrey Eugenides","[Oscar Wilde, Jeffrey Eugenides]",19647,9824.0,1891
95,E.L. James,[E.L. James],25287,25287.0,2012
96,"Bram Stoker, Nina Auerbach, David J. Skal","[Bram Stoker, Nina Auerbach, David J. Skal]",17263,5754.0,1897
97,"Stieg Larsson, Reg Keeland","[Stieg Larsson, Reg Keeland]",31423,15712.0,2006






La funzione **SharedReviewsSum** somma per ogni autore nel dataframe in input, il numero di *TextReviewsSplit* di ogni libro 

In [19]:
def SharedReviewsSum(df):
    '''
    Per ogni autore nel dataframe, somma per ogni libro il numero totale di TextReviewsSplit 
    
    '''
    cont = {}
    for r in df.index:
        ListaAutori = df["AuthorsSplit"][r]
        for aut in ListaAutori:
            if aut in cont:
                cont[aut]+=df["TextReviewsSplit"][r]
            else:
                cont[aut]=df["TextReviewsSplit"][r]
    return pd.DataFrame.from_dict(cont,orient='index',columns=["shared number of reviews with a text"]).sort_values("shared number of reviews with a text",ascending=False)
 

    
    

In [20]:
#Total number of reviews with a text for each author
TextReviewByAuthor=SharedReviewsSum(GoodBooks)
TextReviewByAuthor

,shared number of reviews with a text
Suzanne Collins,340066.0
Stephenie Meyer,258573.0
John Green,230584.0
Veronica Roth,156896.0
Mary GrandPrÃ©,136668.0
...,...
Robert Fagles,1620.0
E.V. Rieu,1620.0
FrÃ©dÃ©ric Mugler,1620.0
Bernard Knox,1620.0


# 
# Question 8: For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

Per ogni anno di pubblicazione viene calcolato per ogni autore il valore totale di "Shared number of reviews with a text", e quindi individuato il massimo

In Alcuni casi, possono esserci più autori che condividono lo stesso valore

In [21]:
Years= list(GoodBooks["original_publication_year"].drop_duplicates().sort_values(ascending=False))
df_years={}
for y in Years:
    AuthorsByYear=SharedReviewsSum(GoodBooks[GoodBooks["original_publication_year"]==y])
    df_years[y]=[AuthorsByYear["shared number of reviews with a text"].max(),AuthorsByYear[AuthorsByYear["shared number of reviews with a text"]==AuthorsByYear["shared number of reviews with a text"].max()].index.values.tolist()]

pd.DataFrame.from_dict(df_years,orient='index',columns=["Largest SharedNumberReviewsWithText","Authors"])


,Largest SharedNumberReviewsWithText,Authors
2015,93600.0,[Paula Hawkins]
2012,140739.0,[John Green]
2011,103489.0,[E.L. James]
2010,96274.0,[Suzanne Collins]
2009,88538.0,[Suzanne Collins]
2008,155254.0,[Suzanne Collins]
2007,51589.0,[Cassandra Clare]
2006,55732.0,[Sara Gruen]
2005,95009.0,[Stephenie Meyer]
2004,30719.0,[Jodi Picoult]


# 
# Question 9: Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets ##

In [22]:
merged=pd.merge(AVGRatingBooks,AVGGoodBooksRatings,left_on="ISBN",right_on="isbn")


In [23]:
merged["Differenza AVG"]=merged["AVG BookRatings"]-merged["AVG GoodBooksRatings"]
merged[['ISBN', 'Book-Title',  'isbn', 'authors','AVG BookRatings',  'AVG GoodBooksRatings',  'Differenza AVG']]

,ISBN,Book-Title,isbn,authors,AVG BookRatings,AVG GoodBooksRatings,Differenza AVG
0,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,043965548X,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",3.53,5.0,-1.47
1,1400032717,The Curious Incident of the Dog in the Night-T...,1400032717,Mark Haddon,4.81,NaN,NaN
2,1594480001,The Kite Runner,1594480001,Khaled Hosseini,2.40,NaN,NaN
3,014038572X,The Outsiders (Now in Speak!),014038572X,S.E. Hinton,4.46,NaN,NaN


# 
# Question 10: Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

La colonna AGE contiene dati mancanti (NaN) e dati errati (sono presenti età superiori ai 150 anni). 


In [24]:
#La colonna AGE contiene errori. Un'eta pari a 244 è evidentemente un errore. 
Users["Age"].max()

244.0

In [25]:
#età massima da considerare per la suddivisione del DF
MaxAge=150

#Gruppi in cui dividere il DataFrame Users
splitdf=list(range(15,MaxAge,10))
splitdf.insert(0,"na")
if splitdf[-1]!=MaxAge:
    splitdf.append(MaxAge)
splitdf.append("more")

#Costruzione Dizionario con gruppi di DF suddivisi per età
dt={}
for elem in splitdf:
    ind = splitdf.index(elem)
    prec = splitdf[ind-1]
    if elem=="na":
        dt[elem]=Users[Users["Age"].isna()]
    elif elem=="more":
        dt[elem]=Users[Users["Age"]>=MaxAge]
    elif elem==15:
        dt[elem]=Users[Users["Age"]<elem]
    else:
        dt[elem]=Users[(Users["Age"]>=prec)&(Users["Age"]<elem)]

      

In [26]:
dt.keys()

dict_keys(['na', 15, 25, 35, 45, 55, 65, 75, 85, 95, 105, 115, 125, 135, 145, 150, 'more'])

In [27]:
#Stampa dei gruppi di users divisi per età
for age in dt:
    AgePrec=splitdf[splitdf.index(age)-1]
    print("\n")
    if age=="na":
        print("Users with unknow age:")     
    elif age=="more":
        print(f"Users with more than {MaxAge} years")
    elif age==15:
        print(f"Users with age 0-{age-1}")
    else:
        print(f"Users with age {AgePrec}-{age-1}")
    print(dt[age][["User-ID","Location","Age"]])
 



Users with unknow age:
        User-ID                            Location  Age
0             1                  nyc, new york, usa  NaN
2             3     moscow, yukon territory, russia  NaN
4             5  farnborough, hants, united kingdom  NaN
6             7                 washington, dc, usa  NaN
7             8            timmins, ontario, canada  NaN
...         ...                                 ...  ...
278849   278850          sergnano, lombardia, italy  NaN
278853   278854               portland, oregon, usa  NaN
278855   278856           brampton, ontario, canada  NaN
278856   278857           knoxville, tennessee, usa  NaN
278857   278858                dublin, n/a, ireland  NaN

[110762 rows x 3 columns]


Users with age 0-14
        User-ID                           Location   Age
10           11     melbourne, victoria, australia  14.0
18           19                          weston, ,  14.0
40           41            santee, california, usa  14.0
119         12

# 
# Question 11 -  Find the books that appear only in the goodbooks datasets.

In [28]:
UniqueGoodBooks=list(set(GoodBooks["isbn"])-set(Books["ISBN"]))

In [29]:
GoodBooks[GoodBooks["isbn"].isin(UniqueGoodBooks)]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,AuthorsSplit,TextReviewsSplit
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,[Suzanne Collins],155254.0
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,"[J.K. Rowling, Mary GrandPrÃ©]",37934.0
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005,Twilight,...,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,[Stephenie Meyer],95009.0
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960,To Kill a Mockingbird,...,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,[Harper Lee],72586.0
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925,The Great Gatsby,...,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,[F. Scott Fitzgerald],51992.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,5297,5297,1858012,2303,375751513,9.780376e+12,"Oscar Wilde, Jeffrey Eugenides",1891,The Picture of Dorian Gray,...,19647,9825,30735,129420,246892,257679,https://images.gr-assets.com/books/1424596966m...,https://images.gr-assets.com/books/1424596966s...,"[Oscar Wilde, Jeffrey Eugenides]",9824.0
95,96,13536860,13536860,18034963,133,345803507,9.780346e+12,E.L. James,2012,Fifty Shades Freed,...,25287,39079,55691,109963,140304,265079,https://images.gr-assets.com/books/1336418837m...,https://images.gr-assets.com/books/1336418837s...,[E.L. James],25287.0
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,"Bram Stoker, Nina Auerbach, David J. Skal",1897,Dracula,...,17263,11756,36719,151396,246043,243825,https://images.gr-assets.com/books/1387151694m...,https://images.gr-assets.com/books/1387151694s...,"[Bram Stoker, Nina Auerbach, David J. Skal]",5754.0
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,"Stieg Larsson, Reg Keeland",2006,Flickan som lekte med elden,...,31423,4623,15781,91021,258236,279592,https://images.gr-assets.com/books/1351778881m...,https://images.gr-assets.com/books/1351778881s...,"[Stieg Larsson, Reg Keeland]",15712.0


# 
# Question 12: Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [30]:
BookCount=Books.groupby(['Book-Title', 'Book-Author'],as_index=False)["ISBN"].count().sort_values("ISBN",ascending=False).rename(columns={"ISBN":"Book Count"})
BookCount

,Book-Title,Book-Author,Book Count
113850,Little Women,Louisa May Alcott,21
248700,Wuthering Heights,Emily Bronte,20
9858,Adventures of Huckleberry Finn,Mark Twain,20
149441,Pride and Prejudice,Jane Austen,18
215213,The Secret Garden,Frances Hodgson Burnett,16
...,...,...,...
86396,Heritage of Hastur,Marion Zimmer Bradley,1
86397,Heritage of Music (4 Volume Set),Michael Raeburn,1
86398,Heritage of Shame,Meg Hutchinson,1
86399,Heritage of Stars,Clifford D. Simak,1


In [31]:
#TOP3 libri che appaiono maggiormente in the Books dataset
BookCount.head(3)

,Book-Title,Book-Author,Book Count
113850,Little Women,Louisa May Alcott,21
248700,Wuthering Heights,Emily Bronte,20
9858,Adventures of Huckleberry Finn,Mark Twain,20


# 
# Question 13: Find the author with the highest average rating according to the goodbooks datasets.

In [32]:
#Lista degli Autori presenti in GoodBooks Dataset
ListaAutori= TextReviewByAuthor.index.values.tolist()

In [33]:
#Calcolo media average rating per ogni autore
AVGRatingAuthors={}
for aut in ListaAutori:
    AVGRatingAuthors[aut]=round(GoodBooks[GoodBooks["authors"].str.contains(aut)]["average_rating"].mean(),2) 
AVGRatingAuthor=pd.DataFrame.from_dict(AVGRatingAuthors,orient='index',columns=["avg rating"]).sort_values("avg rating",ascending=False)
AVGRatingAuthor

,avg rating
Rufus Beck,4.53
Mary GrandPrÃ©,4.50
J.K. Rowling,4.50
Kathryn Stockett,4.45
George R.R. Martin,4.45
...,...
William Shakespeare,3.73
Lauren Weisberger,3.70
Stephenie Meyer,3.66
William Golding,3.64


In [34]:
#Autore/i con AVG Rating più alto
AVGRatingAuthor[AVGRatingAuthor["avg rating"]==AVGRatingAuthor["avg rating"].max()]

,avg rating
Rufus Beck,4.53


In [35]:
#TOP3 Autori con AVG Rating più alto
AVGRatingAuthor.head(3)

,avg rating
Rufus Beck,4.53
Mary GrandPrÃ©,4.50
J.K. Rowling,4.50
